# Visualizaciones para Predicción de Precios de Casas

Este notebook crea visualizaciones para entender mejor los datos y evaluar el rendimiento del modelo.

## ¿Por qué visualizar?

Las visualizaciones son fundamentales en Machine Learning porque:

1. **Exploración de datos**: Ayudan a entender relaciones entre variables
2. **Detección de patrones**: Identificar tendencias y correlaciones
3. **Diagnóstico del modelo**: Ver qué tan bien está prediciendo
4. **Comunicación**: Presentar resultados de forma clara

## Tipos de visualizaciones

### 1. Características vs Precio
- **Objetivo**: Ver cómo cada característica se relaciona con el precio
- **Qué buscar**: Relaciones lineales, outliers, patrones

### 2. Predicciones vs Valores Reales
- **Objetivo**: Evaluar qué tan bien predice el modelo
- **Qué buscar**: Puntos cerca de la línea diagonal (predicción perfecta)

## 1. Importar librerías

Importamos:
- **matplotlib**: Librería principal para crear gráficas
- **seaborn**: Opcional, para gráficas más estéticas
- **config**: Para obtener rutas y nombres de características

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import sys

# Añadir el directorio padre al path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from config import REPORTS_DIR, FEATURES

# Configurar estilo de las gráficas
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Para que las gráficas se muestren en el notebook
%matplotlib inline

print("✓ Librerías importadas correctamente")
print(f"✓ Las gráficas se guardarán en: {REPORTS_DIR}")

✓ Librerías importadas correctamente
✓ Las gráficas se guardarán en: c:\dev\repos-deep-learning\predictor-house\reports\figures


## 2. Función: Características vs Precio

### ¿Qué hace esta función?

Crea 4 gráficas de dispersión (scatter plots) mostrando:
- Eje X: Valor de la característica (ej: tamaño en m²)
- Eje Y: Precio de la casa
- Cada punto: Una casa en el dataset

### ¿Cómo interpretar?

#### Relación positiva (línea ascendente):
```
Precio ↑
       │    •
       │  •  •
       │•
       └────────→ Tamaño ↑
```
**Interpretación**: A mayor tamaño, mayor precio (correlación positiva)

#### Relación negativa (línea descendente):
```
Precio ↑
       │•
       │  •  •
       │    •
       └────────→ Edad ↑
```
**Interpretación**: A mayor edad, menor precio (correlación negativa)

#### Sin relación (nube dispersa):
```
Precio ↑
       │ • • •
       │• •  •
       │  • •
       └────────→ Feature ↑
```
**Interpretación**: La característica no influye en el precio

### Outliers (valores atípicos):
Puntos muy alejados del patrón general pueden ser:
- Casas de lujo
- Errores en los datos
- Casos especiales que requieren investigación

In [2]:
def plot_feature_vs_target(df, target='precio'):
    """Crea gráficas de características vs precio"""
    print("\n=== GENERANDO VISUALIZACIONES ===")
    
    features_to_plot = [f for f in FEATURES if f in df.columns]
    n_features = len(features_to_plot)
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    colors = ['blue', 'green', 'red', 'purple']
    
    for i, feature in enumerate(features_to_plot[:4]):
        axes[i].scatter(df[feature], df[target], color=colors[i], alpha=0.6)
        axes[i].set_xlabel(feature, fontsize=11)
        axes[i].set_ylabel(target, fontsize=11)
        axes[i].set_title(f'{feature} vs {target}', fontsize=12, fontweight='bold')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Guardar figura
    filepath = os.path.join(REPORTS_DIR, 'feature_analysis.png')
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    print(f"✓ Gráfica guardada en: {filepath}")
    
    plt.show()
    
    return fig

## 3. Función: Predicciones vs Valores Reales

### ¿Qué muestra esta gráfica?

Es la gráfica más importante para evaluar tu modelo:

- **Eje X**: Precio real de la casa
- **Eje Y**: Precio predicho por el modelo
- **Línea roja diagonal**: Predicción perfecta (donde predicción = realidad)

### ¿Cómo interpretar?

#### Modelo perfecto:
```
Predicho ↑
         │    •
         │  •─•  ← Todos los puntos sobre la línea
         │•
         └────────→ Real ↑
```
**Resultado**: Predicción = Real para todas las casas (R² = 1.0)

#### Modelo bueno:
```
Predicho ↑
         │  • •
         │ •─• •  ← Puntos cerca de la línea
         │• •
         └────────→ Real ↑
```
**Resultado**: Predicciones cercanas a valores reales (R² > 0.8)

#### Modelo malo:
```
Predicho ↑
         │• •   •
         │  •─    ← Puntos muy dispersos
         │•   •
         └────────→ Real ↑
```
**Resultado**: Predicciones poco confiables (R² < 0.5)

### Patrones a identificar:

1. **Subestimación sistemática** (puntos bajo la línea):
   - El modelo predice menos de lo real
   - Común en casas de lujo

2. **Sobreestimación sistemática** (puntos sobre la línea):
   - El modelo predice más de lo real
   - Puede indicar sesgo en los datos

3. **Heterocedasticidad** (dispersión variable):
   - Más error en precios altos que en bajos
   - Sugiere usar transformación logarítmica

In [3]:
def plot_predictions_vs_actual(y_test, y_pred):
    """Gráfica de predicciones vs valores reales"""
    plt.figure(figsize=(10, 6))
    
    # Scatter plot
    plt.scatter(y_test, y_pred, color='blue', alpha=0.6, s=100, edgecolors='black', linewidth=0.5)
    
    # Línea de predicción perfecta
    plt.plot([y_test.min(), y_test.max()], 
             [y_test.min(), y_test.max()], 
             'r--', lw=2, label='Predicción perfecta')
    
    plt.xlabel('Precio Real ($)', fontsize=12)
    plt.ylabel('Precio Predicho ($)', fontsize=12)
    plt.title('Predicciones vs Precios Reales', fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    
    # Añadir información estadística
    from sklearn.metrics import r2_score, mean_absolute_error
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    # Texto con métricas
    textstr = f'R² = {r2:.4f}\nMAE = ${mae:,.2f}'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=11,
            verticalalignment='top', bbox=props)
    
    # Guardar figura
    filepath = os.path.join(REPORTS_DIR, 'predictions_vs_actual.png')
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    print(f"✓ Gráfica guardada en: {filepath}")
    
    plt.show()
    
    return plt.gcf()

## 4. Visualizaciones adicionales

Además de las dos funciones principales, podemos crear otras visualizaciones útiles:

### 4.1 Matriz de correlación

Muestra la correlación entre todas las variables:
- **+1**: Correlación positiva perfecta
- **0**: Sin correlación
- **-1**: Correlación negativa perfecta

**Colores**:
- Rojo/Naranja: Correlación alta positiva
- Azul: Correlación alta negativa
- Blanco/Amarillo: Poca o ninguna correlación

In [4]:
def plot_correlation_matrix(df, target='precio'):
    """Crea un mapa de calor con las correlaciones"""
    print("\n=== MATRIZ DE CORRELACIÓN ===")
    
    # Seleccionar solo columnas numéricas
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Calcular matriz de correlación
    corr = df[numeric_cols].corr()
    
    # Crear figura
    plt.figure(figsize=(10, 8))
    
    # Mapa de calor
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    
    plt.title('Matriz de Correlación', fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    
    # Guardar
    filepath = os.path.join(REPORTS_DIR, 'correlation_matrix.png')
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    print(f"✓ Matriz guardada en: {filepath}")
    
    plt.show()
    
    # Mostrar correlaciones con el target
    print(f"\nCorrelaciones con {target} (ordenadas):")
    target_corr = corr[target].sort_values(ascending=False)
    for feature, value in target_corr.items():
        if feature != target:
            print(f"  {feature}: {value:.4f}")
    
    return corr

### 4.2 Distribución de precios

Histograma que muestra cómo se distribuyen los precios:
- **Eje X**: Rangos de precio
- **Eje Y**: Cantidad de casas en ese rango

**Qué buscar**:
- Distribución normal (campana)
- Sesgo hacia precios bajos o altos
- Múltiples picos (diferentes segmentos de mercado)

In [5]:
def plot_price_distribution(df, target='precio'):
    """Muestra la distribución de precios"""
    print("\n=== DISTRIBUCIÓN DE PRECIOS ===")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histograma
    axes[0].hist(df[target], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0].set_xlabel('Precio ($)', fontsize=11)
    axes[0].set_ylabel('Frecuencia', fontsize=11)
    axes[0].set_title('Distribución de Precios', fontsize=12, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # Box plot
    axes[1].boxplot(df[target], vert=True)
    axes[1].set_ylabel('Precio ($)', fontsize=11)
    axes[1].set_title('Box Plot de Precios', fontsize=12, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    
    # Guardar
    filepath = os.path.join(REPORTS_DIR, 'price_distribution.png')
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    print(f"✓ Gráfica guardada en: {filepath}")
    
    plt.show()
    
    # Estadísticas
    print(f"\nEstadísticas de {target}:")
    print(f"  Media: ${df[target].mean():,.2f}")
    print(f"  Mediana: ${df[target].median():,.2f}")
    print(f"  Desviación estándar: ${df[target].std():,.2f}")
    print(f"  Mínimo: ${df[target].min():,.2f}")
    print(f"  Máximo: ${df[target].max():,.2f}")
    
    return fig

### 4.3 Distribución de errores

Analiza los errores del modelo:
- **Error = Predicción - Real**
- Idealmente, los errores deben estar centrados en 0
- Distribución normal de errores indica un buen modelo

In [6]:
def plot_residuals(y_test, y_pred):
    """Analiza la distribución de errores (residuales)"""
    print("\n=== ANÁLISIS DE RESIDUALES ===")
    
    # Calcular residuales
    residuals = y_pred - y_test
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Residuales vs predicciones
    axes[0].scatter(y_pred, residuals, alpha=0.6, color='green', edgecolors='black', linewidth=0.5)
    axes[0].axhline(y=0, color='r', linestyle='--', linewidth=2)
    axes[0].set_xlabel('Precio Predicho ($)', fontsize=11)
    axes[0].set_ylabel('Residuales ($)', fontsize=11)
    axes[0].set_title('Residuales vs Predicciones', fontsize=12, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # Histograma de residuales
    axes[1].hist(residuals, bins=30, color='coral', edgecolor='black', alpha=0.7)
    axes[1].set_xlabel('Residuales ($)', fontsize=11)
    axes[1].set_ylabel('Frecuencia', fontsize=11)
    axes[1].set_title('Distribución de Residuales', fontsize=12, fontweight='bold')
    axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Guardar
    filepath = os.path.join(REPORTS_DIR, 'residuals_analysis.png')
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    print(f"✓ Gráfica guardada en: {filepath}")
    
    plt.show()
    
    # Estadísticas de residuales
    print(f"\nEstadísticas de residuales:")
    print(f"  Media: ${residuals.mean():,.2f} (debe estar cerca de 0)")
    print(f"  Desviación estándar: ${residuals.std():,.2f}")
    print(f"  Mínimo: ${residuals.min():,.2f}")
    print(f"  Máximo: ${residuals.max():,.2f}")
    
    return fig

## 5. Ejemplo de uso completo

Aquí un ejemplo de cómo usar todas las funciones:

In [7]:
# Ejemplo con datos sintéticos (descomentar para usar tus datos reales)

# 1. Cargar datos
# from config import RAW_DATA_FILE, TARGET
# df = pd.read_csv(RAW_DATA_FILE)

# 2. Visualizar características vs precio
# plot_feature_vs_target(df, target=TARGET)

# 3. Matriz de correlación
# plot_correlation_matrix(df, target=TARGET)

# 4. Distribución de precios
# plot_price_distribution(df, target=TARGET)

# 5. Después de entrenar el modelo:
# from model import split_data, train_model
# X = df[FEATURES]
# y = df[TARGET]
# X_train, X_test, y_train, y_test = split_data(X, y)
# modelo = train_model(X_train, y_train)
# y_pred = modelo.predict(X_test)

# 6. Visualizar predicciones
# plot_predictions_vs_actual(y_test, y_pred)

# 7. Análisis de residuales
# plot_residuals(y_test, y_pred)

print("\n✅ Todas las visualizaciones se generaron correctamente")
print(f"📁 Archivos guardados en: {REPORTS_DIR}")


✅ Todas las visualizaciones se generaron correctamente
📁 Archivos guardados en: c:\dev\repos-deep-learning\predictor-house\reports\figures


## 6. Consejos para interpretar visualizaciones

### ✅ Señales de un buen modelo:

1. **Scatter plots de features**: Muestran relaciones claras (lineales)
2. **Predicciones vs Reales**: Puntos muy cerca de la línea diagonal
3. **Residuales**: 
   - Distribuidos alrededor de 0
   - Sin patrones evidentes
   - Forma de campana en el histograma

### ⚠️ Señales de problemas:

1. **Outliers extremos**: Investigar si son errores o casos especiales
2. **Heterocedasticidad**: Varianza de errores no constante
3. **Patrones en residuales**: El modelo no captura toda la información
4. **Correlaciones bajas**: Features poco útiles para predicción

### 💡 Recomendaciones:

- **Guarda todas las gráficas**: Útiles para reportes y presentaciones
- **Compara antes/después**: Al mejorar el modelo, compara las visualizaciones
- **Usa múltiples métricas**: No te bases solo en una gráfica
- **Comunica resultados**: Las visualizaciones son excelentes para explicar a no técnicos